In [7]:
import tensorflow as tf
tf.enable_eager_execution()

import pandas as pd
import numpy as np
import os, string, re, csv

import keras2onnx
import onnxruntime

# set seeds for reproducability
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))


In [8]:
idx2char = pd.read_csv("vocab.csv", header=None)
idx2char = idx2char.to_numpy()
vocab = vocab = [x[1] for x in idx2char]

# Creating a mapping from unique characters to indices
char2idx = pd.read_csv("char2idx.csv", header=None)
char2idx = char2idx.to_numpy()

In [9]:
# Creating a mapping from unique characters to indices
char2idx = pd.read_csv("char2idx.csv", header=None)
char2idx = char2idx[0].to_dict()


# The maximum length sentence we want for a single input in characters
seq_length = 100

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [10]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.CuDNNLSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
    return model

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [11]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints_2'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 256)            23296     
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (1, None, 1024)           5251072   
_________________________________________________________________
dense_3 (Dense)              (1, None, 91)             93275     
Total params: 5,367,643
Trainable params: 5,367,643
Non-trainable params: 0
_________________________________________________________________


In [14]:
graph = keras2onnx.convert_keras(model, channel_first_inputs=True)

KeyError: "The name 'BiasAdd_3' refers to an Operation not in the graph."

In [ ]:
dir(model.get_layer(index=1))